# Veri Seti Hakkında Bilgi

In [16]:
import os
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
!pip install researchpy
import researchpy as rp

In [17]:
class Information():
    def __init__(self,data):
        self.data = data
        
    
    def info(self):
        print(data.info())
        
    
    def copy(self):
        data1 = data.copy()
        data2 = data.copy()
    
    def describe(self):
        print(data.describe())
        
        print("**********************************")
        print(" Loan Değişkeninin incelenmesi")
        print("Ortalama: " + str(data["loan"].mean()))
        print("Dolu Gözlem Sayısı: " + str(data["loan"].count())) 
        print("Maksimum Değer: " + str(data["loan"].max()))
        print("Minimum Değer: " + str(data["loan"].min()))
        print("Medyan: " + str(data["loan"].median()))
        print("Standart Sapma: " + str(data["loan"].std()))
        
    def shape(self):
        
        #type(data)
        print(data.axes)
        #print(self.data.axes)
        #print(" ndim = " + str(data.ndim))
        print(" Veri şekli = " + str(data.shape))
        print(" Veri büyüklüğü = " + str(data.size))
        #type(data.values)
    
    def head(self,row = None):
        
        print(data.head(5))
        # print(data.tail(3))
        
    def hedef_degisken(self):
        
        countNo = len(data[data.bad == 0])
        countYes = len(data[data.bad == 1])
        print("Risk barındırmayan müşterilerin yüzdesi : {:.2f}%".format((countNo / (len(data.bad))*100)))
        print("Risk barındıran müşterilerin yüzdesi : {:.2f}%".format((countYes / (len(data.bad))*100)))
        
        print("************************************")
        
        print(data.groupby('bad').mean().T)

# Görselleştirme

In [18]:
class Visualization():
    def __init__(self,data):
        self.data = data
        
    def visual(self):
        data["job"].value_counts().plot.barh().set_title("Job Değişkeninin Sınıf Frekansları");
    
    def countplot(self):
        sns.countplot(x="bad", data=data, palette="bwr")
        plt.show()
        
    def figure(self):
        fig, ax = plt.subplots(1, 2, figsize = (18, 8))
        sns.barplot(x = "job", y = "loan", hue = "bad", data = data, ax = ax[0])
        ax[0].set_ylabel("loan")
        sns.catplot(x = "job", y = "loan", data = data, ax = ax[1]) 
        ax[1].set_ylabel("Loan")
        
    def distplot(self):
        sns.distplot(data.loan, kde = False);
        
    def pairplot(self):
        
        sns.pairplot(data);
        
        
    def pivotTable(self):
        
        print(data.pivot_table("loan",index = "bad",columns = "reason"))
        

# İSTATİSTİK

In [19]:
class stats():
    def __init__(self,data):
        self.data = data
        
    def imports(self):
        
        import scipy.stats as stats
        import statsmodels.stats.api as sms
        import pylab
        from scipy.stats.stats import pearsonr
        import researchpy as rp
        
    
    def table(self):
        
        print(rp.summary_cont(data[["loan","value"]])) 
        
        print("*******************************")
        
        print(rp.summary_cat(data[["bad","job","reason"]]))
        
        print("*******************************")
        
        print(data[["loan","value"]].corr()) # Korelasyon
        
    def shapiro(self):
        
        import scipy.stats as stats
        from scipy.stats import shapiro
        test, p_value = shapiro(data["loan"])
        
        print("Varsayım kontrolünde pvalue = " + str(p_value))
        
        
        if p_value > 0.05:
            print("Örnek dağılımı ile teorik normal dağılım arasında ist. ol. anl. bir fark. yoktur.")
        else:
            print("Örnek dağılımı ile teorik normal dağılım arasında ist. ol. anl. bir fark. vardır.")
        
        test_istatistigi, pvalue = stats.ttest_1samp(data["loan"], popmean = 10000)
        print("pvalue = " + str(pvalue))
        
        if pvalue > 0.05 :
            print("H0 hipotezimiz reddedilemez.")
        else:
            print("H0 hipotezimiz reddelir.")   
        
        
        
    def qqplot(self):
        import pylab
        import scipy.stats as stats
        stats.probplot(data["loan"], dist="norm", plot=pylab)
        pylab.show()
        
    def korelasyon_analiz(self):
        
        # müşteri hakkındaki kötü raporlar arttığında ne olduğunu gözlemlemek istiyorum.
        data.plot.scatter("loan","value");

        print(data["loan"].corr(data["value"])) # parametrik karşılığı
        
        # Hedef değişken BAD için analiz
        
        print(data.corr()['bad'].abs().sort_values(ascending=False))
        
        
    def korelasyon_map(self):
        
        corr = data.corr()
        # print(corr)
        # sns.heatmap(corr, 
        #xticklabels=corr.columns.values,
        #yticklabels=corr.columns.values)
        
        fig,ax = plt.subplots(figsize=(15, 15))
        sns.heatmap(data.corr(), ax=ax, annot=True, linewidths=0.05, fmt= '.2f',cmap="magma")
        plt.show()


# Ön İşleme

In [154]:
class PreProcessing():
    def __init__(self,data):
        self.data = data
        
    
    def outlier(self):
        
        t_data = data.dropna() # Eksik gözlemleri sildik. Aykırı gözlemleri rahatça görebilmek için.
        print(t_data.head())
        
        # Bir tane gözlemimi alıyorum.
        data_loan = data["loan"]
        sns.boxplot(x = data_loan);
        
        Q1 = data_loan.quantile(0.25)
        Q3 = data_loan.quantile(0.75)
        IQR = Q3-Q1
       
        
        print(" IQR = " + str(Q3-Q1))
        print(" Alt Sınır = " + str(Q1- 1.5*IQR))
        print(" Üst Sınır = " + str(Q3 + 1.5*IQR))
        print(data_loan.median())
       
        # Daha sonra aykırı değeri Baskılama ve ya Ortalama yöntemi ile doldurma işlemleri
    
    def MissingValue(self):
        
        print(data.isnull().sum())
        #print(data.isnull())
        #print(data[data.isnull().any(axis = 1)]) # Eksik değer olan satırlara erişmek
        
        # Bir gözlemi örnek alıyorum.
        # data["loan"]
        # data["loan"].mean()
        # print(data["loan"].fillna(data["loan"].mean())) # fillna doldurma işlemi için yapılıyor.
        
        # data.isna().values.any() #Kalan NaN değeri var mı kontrol etmek için
     
    def dropNa(self):
        data.dropna(inplace = True) # tüm eksik gözlemlerin silinmesi
        
     
    
    def fillna(self):
        data.fillna(data.mean()[:],inplace = True)
        
        # kategorik
        
        print(" En sık tekrarlanan sebep:" + str(data1["reason"].mode()[0]))
        print(" En sık tekrarlanan iş:" + str(data1["job"].mode()[0]))
        data1["job"].fillna(data1["job"].mode()[0],inplace = True)
        data1["reason"].fillna(data1["reason"].mode()[0], inplace = True)
        
        
    def scale(self):
        
        # gerekmesi durumunda içerisinde çok büyük değerler olan gözlemler için yapılabilir.
        pass
    
    def normalize(self):
        
        from sklearn import preprocessing
        preprocessing.normalize(X)
        
        
    def dummies(self):
        
        cat = pd.get_dummies(data2[['reason', 'job']])
        X = pd.concat([X, cat[["reason_DebtCon","reason_HomeImp","job_Mgr" ,"job_Office","job_Other","job_ProfEx","job_Sales","job_Self"]]], axis = 1)
        print(X.head)
        
        
    def onhazirlik(self): # burada modele sokulacak target değişken ve çıkarılması gereken değişkenler 
        
        y = data["bad"]
        X = data.drop(["bad","reason","job"],axis = 1)
        
        print(X.head(3))
        
        return y,X
    def train_test_split(self):
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.20, 
                                                    random_state = 42)
        
        
    def drop(self):
        
        # Anlamsız değişkenler ve ya herhangi bir değişkenin çıkarılması işlemi
        pass
        
        
        
        

# Model İşlemleri ve Model Performansını Değerlendirme

In [155]:
# Problemime göre kullanacağım algoritmaları bu fonksiyon altında çağıracağım.
# Regression, Classification, Clustering

class ModelSelect():
    def __init__(self,data,X,y):
        self.data = data
        self.X = X
        self.y = y
        
    def Log(self):
        loj = sm.Logit(y, X)
        loj_model = loj.fit()
        loj_model.summary()
        
    def Sklearn_Log(self):
        
        from sklearn.linear_model import LogisticRegression
        loj = LogisticRegression(solver = "liblinear") # solver = hangi fonksiyon ile işlem yapacaksın? farklı solver'lar ile deneyip bakmak da fayda var.
        loj_model = loj.fit(X,y)
        
        
    def Log_predict(self): # test ve train diye ayırmadan
        
        loj_model.predict_proba(X)[0:10][:,0:2] # OLASILIKLAR
        y_probs = loj_model.predict_proba(X)
        y_probs = y_probs[:,1]
        y_probs[0:10]
        y_pred = [1 if i > 0.3 else 0 for i in y_probs] 
        
    
    def Sklearn_Log_split(self): # train ve test
        
        loj = LogisticRegression(solver = "liblinear")
        loj_model = loj.fit(X_train,y_train)
        # y_pred = loj_model.predict(X_test)
        y_probs = loj_model.predict_proba(X_test)
        y_probs = y_probs[:,1]
        y_probs[0:10]
        y_pred = [1 if i > 0.3 else 0 for i in y_probs]
             
  
    
    def ModelTuning(self):
        
        # GridSearch
        # best parameters
        
        pass
    


# Metrik Değerlendirme

In [25]:
# Model performansını değerlendireceğim class
# MSE,RMSE, f1 score, AUC-CORVE, correlation matrix


class Evaluate():
    def __init__(self,data):
        self.data = data
        
    def acc(self): # split etmeden doğruluk değeri
        
        print( "acc :" + str(accuracy_score(y, y_pred)))
    
    def acc_split(self): # train-test split
        
        print(" train acc : " + str(accuracy_score(y_train, y_pred))) 
        print(" test acc : " + str(accuracy_score(y_test, y_pred))) 
        print(" cross val score : " + str(cross_val_score(loj_model, X_test, y_test, cv = 10).mean())
    
        
        
    def class_report(self):
              
              print(classification_report(y,y_pred))
              
    def class_report_split(self):
              
              print(classification_report(y_test,y_pred))
              
    def confusion_matrix(self):
        
        mtx = confusion_matrix(y, y_pred)
        fig, ax = plt.subplots(figsize=(5,5))
        sns.heatmap(mtx, xticklabels= 'auto', yticklabels='auto', cmap="Blues", annot=True, fmt='d', linewidths=.5,  cbar=False, ax=ax)
        #  square=True,
        plt.ylabel('true label')
        plt.xlabel('predicted label')
              
    def confusion_matrix_split_train(self):
              
              mtx = confusion_matrix(y_train, y_pred)
              fig, ax = plt.subplots(figsize=(5,5))
              sns.heatmap(mtx, xticklabels= 'auto', yticklabels='auto', cmap="Blues", annot=True, fmt='d', linewidths=.5,  cbar=False, ax=ax)
              #  square=True,
              plt.ylabel('true label')
              plt.xlabel('predicted label')
              
              
    def confusion_matrix_split_test(self):
              
              mtx = confusion_matrix(y_test, y_pred)
              fig, ax = plt.subplots(figsize=(5,5))
              sns.heatmap(mtx, xticklabels= 'auto', yticklabels='auto', cmap="Blues", annot=True, fmt='d', linewidths=.5,  cbar=False, ax=ax)
              #  square=True,
              plt.ylabel('true label')
              plt.xlabel('predicted label')
        
        
        
    def auc_roc(self):
              
              print("auc alan değeri:" + str(roc_auc_score(y,y_pred)))
              
              fpr, tpr, thresholds = roc_curve(y, loj_model.predict_proba(X)[:,1])
              plt.figure()
              plt.plot(fpr, tpr, label='AUC (area = %0.2f)' % logit_roc_auc)
              plt.plot([0, 1], [0, 1],'r--')
              plt.xlim([0.0, 1.0])
              plt.ylim([0.0, 1.05])
              plt.xlabel('False Positive Oranı')
              plt.ylabel('True Positive Oranı')
              plt.title('ROC')
              plt.show()
        
    
    def auc_roc_split(self):
              
              print("auc alan değeri:" + str(roc_auc_score(y_test,y_pred)))
              
              fpr, tpr, thresholds = roc_curve(y_test, loj_model.predict_proba(X_test)[:,1])
              plt.figure()
              plt.plot(fpr, tpr, label='AUC (area = %0.2f)' % logit_roc_auc)
              plt.plot([0, 1], [0, 1],'r--')
              plt.xlim([0.0, 1.0])
              plt.ylim([0.0, 1.05])
              plt.xlabel('False Positive Oranı')
              plt.ylabel('True Positive Oranı')
              plt.title('ROC')
              plt.show()
              
              
        
        